In [1]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
!unzip data.zip
!pip install transformers
!pip install datasets

--2021-10-25 17:43:43--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-25 17:43:44--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 212.22K  95.5KB/s    in 2.2s    

2021-10-25 17:43:47 (95.5 KB/s) - ‘data.zip’ saved [217313/217313]

Archive:  data.zip
   creating: data/
   creating: data/train/
  inflating: data/train/train.gold.txt  
  inflating: data/train/train.data.txt  
   crea

In [2]:
## Import relevant packages
import torch
torch.cuda.empty_cache()

import pandas as pd

import random
import numpy as np
from torch.utils.data import Dataset

from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
# from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification , Trainer, TrainingArguments
from datasets import load_metric

rs = 0
torch.manual_seed(rs)
random.seed(rs)
np.random.seed(rs)
torch.backends.cudnn.deterministic = True

# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

**Prepare Data and dump it to CSV**

In [3]:
def prepareDataset():
    x_train = pd.read_csv('/content/data/train/train.data.txt', sep = '\t',header = None, names = ["word","pos","P1P2" , "sentence1", "sentence2"])
    y_train = pd.read_csv('/content/data/train/train.gold.txt', header = None, names = ['labels'])
    y_train = y_train.labels.apply(lambda x: 0 if x == 'F' else 1)
    x_train["sentence1"] = x_train["sentence1"].apply(lambda x: x.lower())
    x_train["sentence2"] = x_train["sentence2"].apply(lambda x: x.lower())
    # x_train['word'] = x_train['word'].apply(lambda x: x.lower())
    # x_train["Sent"] = x_train["sentence1"] + x_train["sep"] + x_train["sentence2"]
    x_train = x_train.join(y_train)
    x_train.to_csv('TrainData.csv')

    x_dev = pd.read_csv('/content/data/validation/validation.data.txt', sep = '\t',header = None, names = ["word","pos","P1P2" , "sentence1", "sentence2"])
    y_dev= pd.read_csv('/content/data/validation/validation.gold.txt', header = None, names = ['labels'])
    y_dev = y_dev.labels.apply(lambda x: 0 if x == 'F' else 1)
    x_dev["sentence1"] = x_dev["sentence1"].apply(lambda x: x.lower())
    x_dev["sentence2"] = x_dev["sentence2"].apply(lambda x: x.lower())
    x_dev = x_dev.join(y_dev)
    x_dev.to_csv('ValidationData.csv')

prepareDataset()


**Load the CSV Files, perform tokenization in Classification Format**
Tokenization: ["Sentence1" + SEP + "Sentence2"]

In [4]:
train_df = pd.read_csv("/content/TrainData.csv",encoding = 'latin-1')
x_train1 = train_df["sentence1"].values.tolist()
x_train2 = train_df["sentence2"].values.tolist()
y_train = train_df["labels"].values.tolist()

val_df = pd.read_csv("/content/ValidationData.csv",encoding = 'latin-1')
x_val1 = val_df["sentence1"].values.tolist()
x_val2 = val_df["sentence2"].values.tolist()
y_val = val_df["labels"].values.tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

x_train_tok = tokenizer(x_train1,x_train2,padding="max_length",truncation=True,max_length = 128)
x_val_tok = tokenizer(x_val1,x_val2,padding="max_length",truncation=True,max_length = 128)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

**Custom Dataset Class**

In [5]:
class DataSet(torch.utils.data.Dataset):
    def __init__(self, tokens, labels=None):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, idx):
        #format as specified by the Hugging face library
        item = {k: torch.tensor(v[idx]) for k, v in self.tokens.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DataSet(x_train_tok, y_train)
val_dataset = DataSet(x_val_tok, y_val)


In [6]:
accuracy = load_metric("accuracy")
def compute_metrics(preds):
    pred_probs, references = preds
    predictions = np.argmax(pred_probs, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=references)
    return acc

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

TrainArgs = TrainingArguments(output_dir = 'models', evaluation_strategy="epoch",num_train_epochs=10,             
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,  
    learning_rate = 5e-5, 
    warmup_steps=200,                
    weight_decay=0.001,    
    logging_steps=100,
    load_best_model_at_end=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_accuracy",
    save_total_limit=1,
    greater_is_better=True)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
trainer = Trainer(model=model,                      
    args=TrainArgs,                  
    train_dataset = train_dataset,         
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics
)
trainer.train()

***** Running training *****
  Num examples = 5428
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6790


Epoch,Training Loss,Validation Loss,Accuracy
1,0.582700,0.610021,0.691223
2,0.373800,0.854687,0.716301
3,0.188400,1.442885,0.688088


***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to models/checkpoint-679
Configuration saved in models/checkpoint-679/config.json
Model weights saved in models/checkpoint-679/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to models/checkpoint-1358
Configuration saved in models/checkpoint-1358/config.json
Model weights saved in models/checkpoint-1358/pytorch_model.bin
Deleting older checkpoint [models/checkpoint-679] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to models/checkpoint-2037
Configuration saved in models/checkpoint-2037/config.json
Model weights saved in models/checkpoint-2037/pytorch_model.bin


KeyboardInterrupt: ignored

In [ ]:
# trainer.evaluate(train_dataset)

In [ ]:
# !zip -r /content/BestModel /content/models/checkpoint-1358

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !mv -f BestModel.zip "drive/MyDrive/NLPA3B"